<a href="https://colab.research.google.com/github/masato-ka/jetracer-circuit-learning/blob/master/train_speed_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive 
drive.mount('/content/drive')

In [0]:
!cp '/content/drive/My Drive/circuit_log_20191202_clean.zip' ./
!unzip -q circuit_log_20191202_clean.zip

In [0]:
import torchvision

transform = torchvision.transforms.Compose([
    torchvision.transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [0]:
import os
import glob
import shutil
import numpy as np
#Adjustment for circuit size and FPS.
NUM_TIMESTEPS = 10
NEW_CIRCUIT_LOG = 'reformat_circuit_log'
CIRCUIT_LOG = 'circuit_log'

os.makedirs(NEW_CIRCUIT_LOG, exist_ok=True)
image_paths = sorted(glob.glob(os.path.join(CIRCUIT_LOG, '*.jpg')))
num_timesteps = NUM_TIMESTEPS
gain = np.exp(-3e-2 * np.linspace(0.0, num_timesteps, num_timesteps))

for idx in range(len(image_paths)-num_timesteps):

    target = np.zeros(num_timesteps)
    label = None
    for i in range(num_timesteps):
        path = os.path.splitext(os.path.basename(image_paths[idx + i]))[0]
        x = float(int(path.split('_')[1]) - 50) / 50.0
        target[i] = x if abs(x) > 0.20 else 0
    label = np.sum(gain * np.abs(target) / np.sum(gain))
    if label is None:
        print("[ERROR] Break! failed create label on " + file_name)
        break
    str_label = str(label).replace('.','-')
    prifix_filename = os.path.basename(image_paths[idx]).split('_')[0]
    rename_file = prifix_filename + '_' + str_label + '.jpg'
    shutil.copy(image_paths[idx], os.path.join(NEW_CIRCUIT_LOG, rename_file))

In [0]:
import torch.utils.data
import os
import PIL.Image
import torch
import glob

class CircuitDataset(torch.utils.data.Dataset):
    def __init__(self, directory, gamma=5e-3, transform=None):
        self.directory = directory
        self.image_paths = sorted(glob.glob(os.path.join(self.directory, '*.jpg')))
        self.transform = transform
        #self.gain = torch.exp(-3e-2*torch.linspace(0.0, num_timesteps, num_timesteps))
        self.color_jitter = torchvision.transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)

    def __len__(self):
        return len(self.image_paths) - NUM_TIMESTEPS
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]        
        image = PIL.Image.open(image_path)
        # if self.transform is not None:
        #     image = self.transform(image)

        image = self.color_jitter(image)
        image = torchvision.transforms.functional.resize(image, (224, 224))
        image = torchvision.transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = torchvision.transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

        slottle = float(os.path.basename(image_path).split('_')[1].split('.')[0].replace('-','.'))
        
        return image, torch.tensor([slottle]).float()


In [0]:
dataset = CircuitDataset(NEW_CIRCUIT_LOG, gamma=1e-2, transform=transform)

In [0]:
import torch
train = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

test = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

In [0]:
import torch 

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 1)

device = torch.device('cuda')
model = model.to(device)

In [0]:
optimizer = torch.optim.Adam(model.parameters())

In [0]:
%load_ext tensorboard
%tensorboard --logdir ./runs

In [0]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
import torch.nn.functional as F

EPOCHS = 10
model.train()
best_epoch = 1e5
for epoch in range(EPOCHS):
    
    epoch_loss = 0.0
    
    for image, target in iter(train):
        image = image.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(image)
        train_loss = F.mse_loss(output, target)
        epoch_loss += float(train_loss)
        train_loss.backward()
        optimizer.step()
        grid = torchvision.utils.make_grid(image)
        writer.add_image('images', grid, 0)
    epoch_loss /= len(train)

    writer.add_scalar('Loss/train',epoch_loss, epoch)
    print('%d: %f' % (epoch, epoch_loss))
    
    if best_epoch > epoch_loss:
        best_epoch = epoch_loss
        torch.save(model.state_dict(), 'circuit_model.pth')

In [0]:
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.transpose((2, 0, 1))
    x = torch.from_numpy(image).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [0]:
import cv2
model.eval()


BASE_SPEED = 0.06
MAX_SPEED = 0.20
MIN_SPEED = 0.01
FPS = 60
size = (224,224)

images = sorted(glob.glob('/content/reformat_circuit_log/*'))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter('output.mp4',fourcc, FPS, size)

def get_throttle(x, accell_work = -15.0, max_speed=0.12, min_speed=0.0, base_speed=0.06):
    x= np.clip(x, 0, 1)
    y = np.exp(accell_work * x); 
    y = (1-y) * (MIN_SPEED) + (MAX_SPEED) * y 
    y = np.clip(y, min_speed, max_speed)
    return y + base_speed


for image in images:
    image = cv2.imread(image)
    x = preprocess(image)
    x = model(x)
    x = x.detach().cpu().float()[0][0].numpy()
    throttle = get_throttle(x, accell_work=-5.0)
    cv2.putText(image, str(throttle), (30,30), cv2.FONT_HERSHEY_SIMPLEX | cv2.FONT_ITALIC, 0.8, (0,0,0), 1, cv2.LINE_AA)
    writer.write(image)
    for i in range(int((0.18-throttle)*100)):
      writer.write(image)
writer.release()



In [0]:
!rm -rf reformat_circuit_log/